In [2]:
from pymongo import MongoClient

from PIL import Image
import cv2
import numpy as np
import warnings
import os

warnings.simplefilter("ignore", Image.DecompressionBombWarning)

from pymongo import MongoClient

# DB 환경 설정을 입력하세요
client = MongoClient("")
db = client['']
collection = db['']

In [3]:
def read_image(path):
    img = Image.open(path)
    img = img.convert('RGB')
    return np.array(img, dtype=np.uint8)

In [4]:
def get_window_size(img):
    height, width, _ = img.shape

    # 적절한 윈도우 크기 계산
    max_width = 1280
    max_height = 1020
    scale = min(max_width / width, max_height / height)
    window_width = int(width * scale)
    window_height = int(height * scale)

    return window_height, window_width

In [5]:
def make_path(document):
    path = os.path.join("/",
                    document["start_path"],
                    document["HDD_name"],
                    document["middle_path"],
                    document["tag"][0],
                    document["file_name"])
    return path

In [10]:
# cursor = collection.find({"label_model":{"$eq":"product"}})
cursor = collection.find({"label_model":{"$eq":"non_product"}})

In [11]:
cv2.namedWindow('image', cv2.WINDOW_NORMAL)

doc = cursor.next()
while doc:
    img_path = make_path(doc)
    img_np = read_image(img_path)
    img_cv = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

    height, width = get_window_size(img_cv)

    cv2.resizeWindow('image', width, height)
    cv2.imshow('image', img_cv)

    k = cv2.waitKey(0)

    if k == ord('d'):
        doc = cursor.next()
    elif k == ord('p'):
        break
    else:
        pass

cv2.destroyAllWindows()